# Example for usage ARc Policy via a Strands Agent

Install requirements

In [1]:
%pip install -r requirements.txt --no-cache-dir --force-reinstall

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 552.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 700.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 751.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 725.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 180.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 242.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 291.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 499.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 133.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 162.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 714.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 706.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 1

In [2]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
# Import libraries
import os
import json
import requests
import boto3
import time
from boto3.session import Session
from strands.tools import tool

# Get boto session
boto_session = Session()

## Setup for the agent

### Create Code for the Agent
Create agents folder if it's not created.

In [2]:
![ ! -d "agents" ] && mkdir agents

In [3]:
%%writefile agents/claimvalidation-agent.py
import os
import logging
import asyncio
import boto3
import json
import uvicorn
from datetime import datetime
from strands import Agent
from strands.models import BedrockModel
from strands.tools import tool
from strands.multiagent.a2a import A2AServer
from fastapi import FastAPI
from strands.hooks import HookProvider, HookRegistry, MessageAddedEvent, BeforeModelCallEvent, BeforeToolCallEvent
from pydantic import BaseModel
from botocore.config import Config as BotocoreConfig
from strands.telemetry import StrandsTelemetry
from findings_utils import extract_reasoning_findings
from strands_tools import retrieve

# Configure the root strands logger
# logging.getLogger("strands").setLevel(logging.DEBUG)
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

# Setup tracing - commented out for now as this adds a lot of trace output that really isn't interesting
# NOTE: To send the OTEL data to an ADOT collector, additional exporter needs to be used
StrandsTelemetry().setup_console_exporter()

# AWS Configuration
AWS_REGION = "us-east-1"
# MODEL_ID = "anthropic.claude-sonnet-4-5-20250929-v1:0"

# Initialize Bedrock client to verify connectivity
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=AWS_REGION
)

print(f"✓ AWS Bedrock configured for region: {AWS_REGION}")
# print(f"✓ Using model: {MODEL_ID}")

# Supply the pre-installed polciy and guardrail IDs
# ARC_POLICY_ARN = "arn:aws:bedrock:us-east-1:505766489067:automated-reasoning-policy/413037g0my75"
# GUARDRAIL_ID = "aq9ga2k7be2c"
# GUARDRAIL_VERSION = "DRAFT"
# KNOWLEDGE_BASE_ID = "2KSYYY00BD"

ARC_POLICY_ARN = "arn:aws:bedrock:us-east-1:161615149547:automated-reasoning-policy/malxiyr0ojy2"
GUARDRAIL_ID = "an852wptcjol"
GUARDRAIL_VERSION = "4"
KNOWLEDGE_BASE_ID = "CZDJXI9C4E"

# NOTE: the default model for Strands is us.anthropic.claude-sonnet-4-20250514-v1:0

# Setup the environment for the agent and tool
# Allow for the metadata to be retrieved on sources from the KB
os.environ['RETRIEVE_ENABLE_METADATA_DEFAULT'] = 'true'
# Allow for the retrieve tool to interact with the KB
os.environ['KNOWLEDGE_BASE_ID'] = KNOWLEDGE_BASE_ID

# Define a notification hook to listen to events and then process the result and call
# Automated Reasoning attached via the Guardrail and report on the findings.  This
# can be used possibly re-write the output or add a flag on if the output is correct.
# Define a notification hook to listen to events and then process the result and call
# Automated Reasoning attached via the Guardrail and report on the findings.  This
# can be used possibly re-write the output or add a flag on if the output is correct.
class NotifyOnlyGuardrailsHook(HookProvider):
    
    def __init__(self, guardrail_id: str, guardrail_version: str, arc_policy_arn: str):
        self.guardrail_id = guardrail_id
        self.guardrail_version = guardrail_version
        self.arc_policy_arn = arc_policy_arn
        self.bedrock_client = boto3.client("bedrock-runtime")
        self.input = ''
        self.claim_valid = True
        self.findings = ''
        self.policy_definition = {}
        self.before_tool_event_flag = False
        self.before_model_event_flag = False

        if self.arc_policy_arn:
            try:
                bedrock_client = boto3.client('bedrock')
                response = bedrock_client.export_automated_reasoning_policy_version(policyArn=self.arc_policy_arn)
                self.policy_definition = response.get('policyDefinition', {})
            except Exception as e:
                print(f"Error getting policy definition: {str(e)}")
                raise

    def register_hooks(self, registry: HookRegistry) -> None:
        registry.add_callback(BeforeModelCallEvent, self.before_model_event)
        registry.add_callback(BeforeToolCallEvent, self.before_tool_event)
        registry.add_callback(MessageAddedEvent, self.message_added)

    def message_added(self, event: MessageAddedEvent) -> None:
        if self.before_tool_event_flag:
            # Since a tool was called, just ignore this message addition
            self.before_tool_event_flag = False
            return
        
        # Get the content
        content = "".join(block.get("text", "") for block in event.message.get("content", []))

        # Determine the source
        if event.message.get("role") == "user":
            # Store the input for later usage and allow the loop to continue to process
            self.input = content
            return

        if not content:
            return
            #do something 

        # Capture if this is the first time that findings will be created
        first_findings = (not self.findings)

        # Format a request to send to the guardrail
        content_to_validate = [
            {"text": {"text": self.input, "qualifiers": ["query"]}},
            {"text": {"text": content, "qualifiers": ["guard_content"]}}
        ]
        
        # Call the guardrail
        response = self.bedrock_client.apply_guardrail(
            guardrailIdentifier=self.guardrail_id,
            guardrailVersion=self.guardrail_version,
            source="OUTPUT",
            content=content_to_validate
        )

        # Determine if the output is correct
        self.findings = extract_reasoning_findings(response, self.policy_definition)
         
        assessments = response.get("assessments", [])
        if assessments and len(assessments):
            self.claim_valid = False

        # Add information to the output
        if self.findings and first_findings:
            new_output = content
            new_output = new_output + f"\n*** FINDINGS: ***:\n{self.findings}"
            new_output = new_output + f"\n*** CLAIM VALID: ***:\n{self.claim_valid}"
            event.message["content"][0]["text"] = new_output
        
    def before_model_event(self, event: BeforeModelCallEvent) -> None:
        self.before_model_event_flag = True

    def before_tool_event(self, event: BeforeToolCallEvent) -> None:
        self.before_tool_event_flag = True

# Create structured output
class StructuredOutputModel(BaseModel):
    claim_valid: bool
    content: str
    findings: str

agent_instructions="""You are an expert automotive claims validaiton specialist that determines if the users auto insurance claim is valid based on the provided information and details within the policy contract.
    
You will be provided with JSON data that has claim information and vehicle damage information, you should:
1. Extract from the JSON data required claim information to be validated
2. Focus on time of event and time of claim creation
3. Focus on claims and coverage inconsistencies

Your responses should :
- If your response is "Valid claim", then output the provide the full JSON structure provided from the input
- If you response is "Invalid claim", then output the response "This Claim is Invalid and no appraisal nor settlement is required"
- If you response is "Invalid claim", then provide clear explanation on why is invalid in the output
- In cases where a clear outcome is not present, recommend the user to check with their insurance agent directly. 

Take your time to think though the answer and evalute carefully."

"""

# Create agent with the guardrail-protected model
agent = Agent(
    name="Claims Validator - ARC",
    description="A Single agent with Claims Validation tools capabilities",
    hooks=[NotifyOnlyGuardrailsHook(GUARDRAIL_ID, GUARDRAIL_VERSION, ARC_POLICY_ARN)],
    tools=[retrieve],
    system_prompt=agent_instructions
)

################# A2A ################
app = FastAPI()
runtime_url = os.environ.get('AGENTCORE_RUNTIME_URL', 'http://127.0.0.1:9000/')
host, port = "0.0.0.0", 9000

a2a_server = A2AServer(
    agent=agent,
    http_url=runtime_url,
    serve_at_root=True,
)

# Respond to pings
@app.get("/ping")
def ping():
    return {"status": "healthy"}

# Any request, call the server
app.mount("/", a2a_server.to_fastapi_app())

# If this is running from main, start the server
if __name__ == "__main__":
    uvicorn.run(app, host=host, port=port)

################# A2A ################

Overwriting agents/claimvalidation-agent.py


Let's write a requirements.txt file with dependencies that are needed for the agent.

In [4]:
%%writefile agents/requirements.txt
boto3
botocore
bedrock-agentcore
bedrock-agentcore-starter-toolkit
strands-agents
strands-agents[a2a]
strands-agents-tools
pyyaml
ddgs
PyYAML
PyPDF2
opentelemetry-sdk
opentelemetry-exporter-otlp
opentelemetry-instrumentation-boto
opentelemetry-sdk-extension-aws
pydantic 
httpx
fastapi
uvicorn[standard]
requests

Overwriting agents/requirements.txt


###  Deploy to AgentCore Runtime

#### Setup Cognito User Pool

In [5]:
from helpers.utils import setup_cognito_user_pool, reauthenticate_user

print("Setting up Amazon Cognito user pool...")
cognito_config = (
    setup_cognito_user_pool()
)  # You'll get your bearer token from this output cell.
print("Cognito setup completed ✓")

Setting up Amazon Cognito user pool...
Pool id: us-east-1_hQnTGy6rJ
Discovery URL: https://cognito-idp.us-east-1.amazonaws.com/us-east-1_hQnTGy6rJ/.well-known/openid-configuration
Client ID: 5h0pvocld3brttgeci9n4s1m0u
Bearer Token: eyJraWQiOiJcL2FvVFNqdnpcL1VPdzd5anRVbFBNdlhqUng5VU1raTRCK0pFUlYySFBHNEU9IiwiYWxnIjoiUlMyNTYifQ.eyJzdWIiOiI0NGQ4NDQwOC1lMGMxLTcwZmYtZjVjMy05N2Y0NzZmYWVkY2UiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtZWFzdC0xLmFtYXpvbmF3cy5jb21cL3VzLWVhc3QtMV9oUW5UR3k2ckoiLCJjbGllbnRfaWQiOiI1aDBwdm9jbGQzYnJ0dGdlY2k5bjRzMW0wdSIsIm9yaWdpbl9qdGkiOiI5NmIzOGMzMS0yOWRlLTRiYWItYTFlNC1iNDE4Mjc2NWNhZWQiLCJldmVudF9pZCI6IjQ5ZWIzOTJmLWY0ZmMtNDcwZi04Nzg5LWMxMGQ0MjJhNjJjMSIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoiYXdzLmNvZ25pdG8uc2lnbmluLnVzZXIuYWRtaW4iLCJhdXRoX3RpbWUiOjE3NzExOTI1NzgsImV4cCI6MTc3MTE5NjE3OCwiaWF0IjoxNzcxMTkyNTc4LCJqdGkiOiJiZGNjMmExYi1iNzc3LTQ3ZTMtYjIzOS0yM2Q0MDliODJkYzMiLCJ1c2VybmFtZSI6InRlc3R1c2VyIn0.SrYQWllnyaBBkUn6fV0Wphv7gpb57o6KNAd25jbznJXXaB9Weut17Xorm-iJoc6Wy-Gozj5ym

#### Create IAM Role for the Agents

In [6]:
from helpers.utils import create_agentcore_runtime_execution_role, AWS_CLAIMSVALIDATION_ROLE_NAME

execution_role_arn = create_agentcore_runtime_execution_role(AWS_CLAIMSVALIDATION_ROLE_NAME)

ℹ️ Role AWSClaimsValidationAssistantBedrockAgentCoreRole-us-east-1 already exists
Role ARN: arn:aws:iam::161615149547:role/AWSClaimsValidationAssistantBedrockAgentCoreRole-us-east-1


In [7]:
execution_role_arn

'arn:aws:iam::161615149547:role/AWSClaimsValidationAssistantBedrockAgentCoreRole-us-east-1'

##### Configure and deploy our  agent:

In [8]:
from bedrock_agentcore_starter_toolkit import Runtime

agentcore_runtime_claimvalidation_agent = Runtime()
claimvalidation_agent_name="aws_claimvalidation_assistant"

region = boto_session.region_name

# Configure the deployment
response_claimvalidation_agent = agentcore_runtime_claimvalidation_agent.configure(
    entrypoint="agents/claimvalidation-agent.py",
    execution_role=execution_role_arn,
    auto_create_ecr=True,
    requirements_file="agents/requirements.txt",
    region=region,
    agent_name=claimvalidation_agent_name,
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [cognito_config.get("client_id")],
            "discoveryUrl": cognito_config.get("discovery_url"),
        }
    },
    protocol="A2A",
)

print("Configuration completed:", response_claimvalidation_agent)




Entrypoint parsed: file=/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/agents/claimvalidation-agent.py, bedrock_agentcore_name=claimvalidation-agent
Memory disabled - agent will be stateless
Configuring BedrockAgentCore agent: aws_claimvalidation_assistant


💡 No container engine found (Docker/Finch/Podman not installed)

✓ Default deployment uses CodeBuild (no container engine needed), For local builds, install Docker, Finch, or 
Podman

Memory disabled
Network mode: PUBLIC


⚠️ Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64', so local builds
won't work.
Please use default launch command which will do a remote cross-platform build using code build.For deployment other
options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

📄 Using existing Dockerfile: 
/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/Dockerfile

Generated .dockerignore: /home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.dockerignore
Changing default agent from 'no_A2S_claimvalidation_assistant' to 'aws_claimvalidation_assistant'
Bedrock AgentCore configured: /home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.bedrock_agentcore.yaml


Configuration completed: config_path=PosixPath('/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.bedrock_agentcore.yaml') dockerfile_path=PosixPath('/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/Dockerfile') dockerignore_path=PosixPath('/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.dockerignore') runtime='None' runtime_type=None region='us-east-1' account_id='161615149547' execution_role='arn:aws:iam::161615149547:role/AWSClaimsValidationAssistantBedrockAgentCoreRole-us-east-1' ecr_repository=None auto_create_ecr=True s3_path=None auto_create_s3=False memory_id=None network_mode='PUBLIC' network_subnets=None network_security_groups=None network_vpc_id=None


In [9]:
launch_result_claimvalidation = agentcore_runtime_claimvalidation_agent.launch()
print("Launch completed:", launch_result_claimvalidation.agent_arn)

claimvalidation_agent_arn = launch_result_claimvalidation.agent_arn

🚀 Launching Bedrock AgentCore (cloud mode - RECOMMENDED)...
   • Deploy Python code directly to runtime
   • No Docker required (DEFAULT behavior)
   • Production-ready deployment

💡 Deployment options:
   • runtime.launch()                → Cloud (current)
   • runtime.launch(local=True)      → Local development
Memory disabled - skipping memory creation
Starting CodeBuild ARM64 deployment for agent 'aws_claimvalidation_assistant' to account 161615149547 (us-east-1)
Generated image tag: 20260215-215623-122
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: aws_claimvalidation_assistant
ECR repository available: 161615149547.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-aws_claimvalidation_assistant
Using execution role from config: arn:aws:iam::161615149547:role/AWSClaimsValidationAssistantBedrockAgentCoreRole-us-east-1
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent

✅ Reusing existing ECR repository: 161615149547.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-aws_claimvalidation_assistant


Reusing existing CodeBuild execution role: arn:aws:iam::161615149547:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-1db4e090aa
Using dockerignore.template with 47 patterns for zip filtering
Uploaded source to S3: aws_claimvalidation_assistant/source.zip
Updated CodeBuild project: bedrock-agentcore-aws_claimvalidation_assistant-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.0s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 8.2s
🔄 DOWNLOAD_SOURCE started (total: 9s)
✅ DOWNLOAD_SOURCE completed in 2.1s
🔄 BUILD started (total: 11s)
✅ BUILD completed in 18.6s
🔄 POST_BUILD started (total: 30s)
✅ POST_BUILD completed in 14.5s
🔄 FINALIZING started (total: 44s)
✅ FINALIZING completed in 1.0s
🔄 COMPLETED started (total: 45s)
✅ COMPLETED completed in 1.0s
🎉 CodeBuild completed successfully in 0m 46s
CodeBuild completed successfully
CodeBuild project configuration saved
Dep

Launch completed: arn:aws:bedrock-agentcore:us-east-1:161615149547:runtime/aws_claimvalidation_assistant-JahQJ5DGNQ


In [10]:
status_response = agentcore_runtime_claimvalidation_agent.status()
status = status_response.endpoint["status"]

print(f"Final status: {status}")

Retrieved Bedrock AgentCore status for: aws_claimvalidation_assistant


Final status: READY


#### Export and save outputs

Export variables to be used in next notebooks:

In [11]:
CLAIMSVALIDATION_AGENT_ID = launch_result_claimvalidation.agent_id
CLAIMSVALIDATION_AGENT_ARN = launch_result_claimvalidation.agent_arn
CLAIMSVALIDATION_AGENT_NAME = claimvalidation_agent_name


CLAIMSVALIDATION_COGNITO_CLIENT_ID = cognito_config.get("client_id")
CLAIMSVALIDATION_COGNITO_SECRET = cognito_config.get("client_secret")
CLAIMSVALIDATION_DISCOVERY_URL = cognito_config.get("discovery_url")

%store CLAIMSVALIDATION_AGENT_ID
%store CLAIMSVALIDATION_AGENT_ARN
%store CLAIMSVALIDATION_AGENT_NAME
%store CLAIMSVALIDATION_COGNITO_CLIENT_ID
%store CLAIMSVALIDATION_COGNITO_SECRET
%store CLAIMSVALIDATION_DISCOVERY_URL

Stored 'CLAIMSVALIDATION_AGENT_ID' (str)
Stored 'CLAIMSVALIDATION_AGENT_ARN' (str)
Stored 'CLAIMSVALIDATION_AGENT_NAME' (str)
Stored 'CLAIMSVALIDATION_COGNITO_CLIENT_ID' (str)
Stored 'CLAIMSVALIDATION_COGNITO_SECRET' (str)
Stored 'CLAIMSVALIDATION_DISCOVERY_URL' (str)


In [12]:
from helpers.utils import put_ssm_parameter, SSM_CLAIMSVALIDATION_AGENT_ARN

put_ssm_parameter(SSM_CLAIMSVALIDATION_AGENT_ARN, claimvalidation_agent_arn)

### Invoking A2A agents

Firstly, let's refresh the auth token:

In [22]:
bearer_token = reauthenticate_user(
    cognito_config.get("client_id"), 
    cognito_config.get("client_secret")
)

In [23]:
cognito_config.get("client_id")

'5h0pvocld3brttgeci9n4s1m0u'

In [24]:
CLAIMSVALIDATION_COGNITO_CLIENT_ID

'5h0pvocld3brttgeci9n4s1m0u'

In [25]:
from uuid import uuid4
from urllib.parse import quote

def fetch_agent_card(agent_arn):
    # URL encode the agent ARN
    escaped_agent_arn = quote(agent_arn, safe='')

    # Construct the URL
    url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations/.well-known/agent-card.json"
    
    # Generate a unique session ID
    session_id = str(uuid4())
    print(f"Generated session ID: {session_id}")

    # Set headers
    headers = {
        'Accept': '*/*',
        'Authorization': f'Bearer {bearer_token}',
        'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id,
    }

    try:
        # Make the request
        response = requests.get(url, headers=headers, timeout=900)
        response.raise_for_status()

        # Parse and pretty print JSON
        agent_card = response.json()
        print(json.dumps(agent_card, indent=2))

        return agent_card

    except requests.exceptions.RequestException as e:
        print(f"Error fetching agent card: {e}")
        raise e

In [26]:
fetch_agent_card(claimvalidation_agent_arn)

Generated session ID: 23688d65-2e95-4a71-8a65-5addc4e7021a
{
  "capabilities": {
    "streaming": true
  },
  "defaultInputModes": [
    "text"
  ],
  "defaultOutputModes": [
    "text"
  ],
  "description": "A Single agent with Claims Validation tools capabilities",
  "name": "Claims Validator - ARC",
  "preferredTransport": "JSONRPC",
  "protocolVersion": "0.3.0",
  "skills": [
    {
      "description": "Retrieves knowledge based on the provided text from Amazon Bedrock Knowledge Bases.\n\nKey Features:\n1. Semantic Search:\n   - Vector-based similarity matching\n   - Relevance scoring (0.0-1.0)\n   - Score-based filtering\n\n2. Advanced Configuration:\n   - Custom result limits\n   - Score thresholds\n   - Regional support\n   - Multiple knowledge bases\n\n3. Response Format:\n   - Sorted by relevance\n   - Includes metadata\n   - Source tracking\n   - Score visibility\n\n4. Example Response:\n   {\n     \"content\": {\n       \"text\": \"Document content...\",\n       \"type\": \"

{'capabilities': {'streaming': True},
 'defaultInputModes': ['text'],
 'defaultOutputModes': ['text'],
 'description': 'A Single agent with Claims Validation tools capabilities',
 'name': 'Claims Validator - ARC',
 'preferredTransport': 'JSONRPC',
 'protocolVersion': '0.3.0',
 'skills': [{'description': 'Retrieves knowledge based on the provided text from Amazon Bedrock Knowledge Bases.\n\nKey Features:\n1. Semantic Search:\n   - Vector-based similarity matching\n   - Relevance scoring (0.0-1.0)\n   - Score-based filtering\n\n2. Advanced Configuration:\n   - Custom result limits\n   - Score thresholds\n   - Regional support\n   - Multiple knowledge bases\n\n3. Response Format:\n   - Sorted by relevance\n   - Includes metadata\n   - Source tracking\n   - Score visibility\n\n4. Example Response:\n   {\n     "content": {\n       "text": "Document content...",\n       "type": "TEXT"\n     },\n     "location": {\n       "customDocumentLocation": {\n         "id": "document_id"\n       },\n 

#### Test agents

Now, let's invoke the first agent, using A2A:

In [27]:
import asyncio
import logging
import os
from uuid import uuid4

import httpx
from a2a.client import A2ACardResolver, ClientConfig, ClientFactory
from a2a.types import Message, Part, Role, TextPart

logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

DEFAULT_TIMEOUT = 300  # set request timeout to 5 minutes

def format_agent_response(response):
    """Extract and format agent response for human readability."""
    # Get the main response text from artifacts
    if response.artifacts and len(response.artifacts) > 0:
        artifact = response.artifacts[0]
        if artifact.parts and len(artifact.parts) > 0:
            return artifact.parts[0].root.text
    
    # Fallback: concatenate all agent messages from history
    agent_messages = [
        msg.parts[0].root.text 
        for msg in response.history 
        if msg.role.value == 'agent' and msg.parts
    ]
    return ''.join(agent_messages)


def create_message(*, role: Role = Role.user, text: str) -> Message:
    return Message(
        kind="message",
        role=role,
        parts=[Part(TextPart(kind="text", text=text))],
        message_id=uuid4().hex,
    )

async def send_sync_message(agent_arn, message: str):
    # Get runtime URL from environment variable
    escaped_agent_arn = quote(agent_arn, safe='')

    # Construct the URL
    runtime_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations/"
    
    # Generate a unique session ID
    session_id = str(uuid4())
    print(f"Generated session ID: {session_id}")

    # Add authentication headers for AgentCore
    headers = {"Authorization": f"Bearer {bearer_token}",
              'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id}
        
    async with httpx.AsyncClient(timeout=DEFAULT_TIMEOUT, headers=headers) as httpx_client:
        # Get agent card from the runtime URL
        resolver = A2ACardResolver(httpx_client=httpx_client, base_url=runtime_url)
        agent_card = await resolver.get_agent_card()
        print(agent_card)

        # Agent card contains the correct URL (same as runtime_url in this case)
        # No manual override needed - this is the path-based mounting pattern

        # Create client using factory
        config = ClientConfig(
            httpx_client=httpx_client,
            streaming=False,  # Use non-streaming mode for sync response
        )
        factory = ClientFactory(config)
        client = factory.create(agent_card)

        # Create and send message
        msg = create_message(text=message)

        print(msg)

        # With streaming=False, this will yield exactly one result
        async for event in client.send_message(msg):
            if isinstance(event, Message):
                logger.info(event.model_dump_json(exclude_none=True, indent=2))
                return event
            elif isinstance(event, tuple) and len(event) == 2:
                # (Task, UpdateEvent) tuple
                task, update_event = event
                logger.info(f"Task: {task.model_dump_json(exclude_none=True, indent=2)}")
                if update_event:
                    logger.info(f"Update: {update_event.model_dump_json(exclude_none=True, indent=2)}")
                return task
            else:
                # Fallback for other response types
                logger.info(f"Response: {str(event)}")
                return event

In [28]:
result = await send_sync_message(claimvalidation_agent_arn, "I was delivering goods for my company using my personal car. I got into an accident, this was over 3 months ago. Does my policy cover this?")
formatted_output = format_agent_response(result)
print(formatted_output)

Generated session ID: cc3af828-e159-48db-aa7b-a0509876ed4d
additional_interfaces=None capabilities=AgentCapabilities(extensions=None, push_notifications=None, state_transition_history=None, streaming=True) default_input_modes=['text'] default_output_modes=['text'] description='A Single agent with Claims Validation tools capabilities' documentation_url=None icon_url=None name='Claims Validator - ARC' preferred_transport='JSONRPC' protocol_version='0.3.0' provider=None security=None security_schemes=None signatures=None skills=[AgentSkill(description='Retrieves knowledge based on the provided text from Amazon Bedrock Knowledge Bases.\n\nKey Features:\n1. Semantic Search:\n   - Vector-based similarity matching\n   - Relevance scoring (0.0-1.0)\n   - Score-based filtering\n\n2. Advanced Configuration:\n   - Custom result limits\n   - Score thresholds\n   - Regional support\n   - Multiple knowledge bases\n\n3. Response Format:\n   - Sorted by relevance\n   - Includes metadata\n   - Source t